In [2]:
import pandas as pd
import math
import keras
from keras.layers import Dense,Embedding, Flatten, Conv1D, GlobalMaxPooling1D, LSTM, Bidirectional
from keras.preprocessing.text import text_to_word_sequence,Tokenizer
from keras.preprocessing.sequence import pad_sequences

C:\Users\David\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
df = pd.read_pickle("./cryptic_dataset/combined_fifteen_times_final.pickle").drop_duplicates()

In [4]:
cc_types = "is_anagram	is_homophone	is_double	is_cryptic	is_contain	is_reverse	is_alternate	is_init	is_delete	is_charade	is_&lit	is_hidden	is_spoonerism	is_palindrome".split('	')

In [5]:
# df.clue = df.clue.apply(text_to_word_sequence)

In [6]:
cc_types_dfs = [df[df[cc_type]==True] for cc_type in cc_types]

In [7]:
for cc_type,cc_type_df in zip(cc_types,cc_types_dfs):
    cc_type_df['category'] = cc_type

C:\Users\David\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [8]:
concatenated_df = pd.concat(cc_types_dfs)

In [9]:
max_size = concatenated_df.groupby('category').count().max()[0]

In [10]:
lst = [concatenated_df]
for class_index, group in concatenated_df.groupby('category'):
    sample = group.sample(max_size-len(group), replace=True, )
    lst.append(sample)
upsampled_df = pd.concat(lst)

In [11]:
upsampled_df.groupby('category').count()

,clue,exp,is_anagram,is_homophone,is_double,is_cryptic,is_contain,is_reverse,is_alternate,is_init,is_delete,is_charade,is_&lit,is_hidden,is_spoonerism,is_palindrome
category,,,,,,,,,,,,,,,,
is_&lit,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581
is_alternate,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581
is_anagram,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581
is_charade,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581
is_contain,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581
is_cryptic,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581
is_delete,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581
is_double,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581
is_hidden,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581,36581


In [12]:
upscaled_cc_dfs = [x[1] for x in upsampled_df.groupby('category')]

In [13]:
def get_input_val_test(df):
    length = len(df)
    input_len = math.floor(length*0.7)
    val_len  = math.floor(length*0.2)
    test_len = math.floor(length*0.1)
    input_df = df[:input_len]
    val_df = df[input_len:input_len+val_len]
    test_df = df[input_len+val_len:]
    return input_df,val_df,test_df

In [14]:
upsampled_cc_types_dfs = [x[1] for x in upsampled_df.groupby('category')]

In [15]:
input_dfs = []
val_dfs = []
test_dfs = []
for cc_type_df in upsampled_cc_types_dfs:
    input_df,val_df,test_df = get_input_val_test(cc_type_df)
    input_dfs.append(input_df)
    val_dfs.append(val_df)
    test_dfs.append(test_df)

In [130]:
cc_input_df = pd.concat(input_dfs).drop('category',axis=1)
cc_val_df = pd.concat(val_dfs).drop('category',axis=1).drop_duplicates()
cc_test_df = pd.concat(test_dfs).drop('category',axis=1).drop_duplicates()

In [131]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df.clue.tolist())
cc_input_data = pad_sequences(tokenizer.texts_to_sequences(cc_input_df.clue.tolist()),maxlen=15)
cc_val_data = pad_sequences(tokenizer.texts_to_sequences(cc_val_df.clue.tolist()),maxlen=15)
cc_test_data = pad_sequences(tokenizer.texts_to_sequences(cc_test_df.clue.tolist()),maxlen=15)

In [132]:
cc_input_data_out = cc_input_df[cc_input_df.columns[2:]] * 1
cc_val_data_out = cc_val_df[cc_val_df.columns[2:]] * 1
cc_test_data_out = cc_test_df[cc_test_df.columns[2:]] * 1

In [133]:
vocab_size = len(tokenizer.word_index) + 1

In [134]:
model = keras.Sequential()
# model.add(Embedding(vocab_size, 300))
model.add(Dense(14, activation='relu', input_dim=15))
model.add(Dense(14, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['categorical_accuracy'])

In [135]:
history = model.fit(cc_input_data,cc_input_data_out ,validation_data=(cc_val_data,cc_val_data_out), batch_size=128, epochs=6)

Train on 358484 samples, validate on 41573 samples
Epoch 1/6
358484/358484 [==============================] - 10s 27us/step - loss: 2.9708 - categorical_accuracy: 0.1521 - val_loss: 2.1487 - val_categorical_accuracy: 0.2142
Epoch 2/6
358484/358484 [==============================] - 9s 25us/step - loss: 2.2879 - categorical_accuracy: 0.1800 - val_loss: 2.1582 - val_categorical_accuracy: 0.2312
Epoch 3/6
358484/358484 [==============================] - 9s 26us/step - loss: 2.2866 - categorical_accuracy: 0.1823 - val_loss: 2.1418 - val_categorical_accuracy: 0.2313
Epoch 4/6
358484/358484 [==============================] - 10s 27us/step - loss: 2.2837 - categorical_accuracy: 0.1857 - val_loss: 2.1385 - val_categorical_accuracy: 0.2394
Epoch 5/6
358484/358484 [==============================] - 10s 27us/step - loss: 2.2817 - categorical_accuracy: 0.1891 - val_loss: 2.1392 - val_categorical_accuracy: 0.2395
Epoch 6/6
358484/358484 [==============================] - 9s 26us/step - loss: 2.2802

In [307]:
model.evaluate(x=cc_test_data, y=cc_test_data_out, batch_size=None, verbose=1, sample_weight=None, steps=None)

30302/30302 [==============================] - 0s 16us/step


[0.3701306075559938, 0.2917629199441955]

In [217]:
model = keras.Sequential()
# model.add(Embedding(vocab_size, 300,input_length=15))
model.add(Dense(300, activation='relu', input_dim=15))
model.add(Dense(14, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

In [218]:
history = model.fit(cc_input_data,cc_input_data_out ,validation_data=(cc_val_data,cc_val_data_out), batch_size=128, epochs=4)

Train on 711924 samples, validate on 42542 samples
Epoch 1/4
711924/711924 [==============================] - 11s 16us/step - loss: 2.7505 - categorical_accuracy: 0.3063 - val_loss: 2.6746 - val_categorical_accuracy: 0.2312
Epoch 2/4
711924/711924 [==============================] - 10s 15us/step - loss: 2.7355 - categorical_accuracy: 0.3131 - val_loss: 2.6953 - val_categorical_accuracy: 0.2227
Epoch 3/4
711924/711924 [==============================] - 10s 15us/step - loss: 2.7349 - categorical_accuracy: 0.3134 - val_loss: 2.6953 - val_categorical_accuracy: 0.2227
Epoch 4/4
711924/711924 [==============================] - 10s 15us/step - loss: 2.7349 - categorical_accuracy: 0.3134 - val_loss: 2.6953 - val_categorical_accuracy: 0.2227


In [216]:
model.evaluate(x=cc_test_data, y=cc_test_data_out, batch_size=None, verbose=1, sample_weight=None, steps=None)

16968/16968 [==============================] - 0s 17us/step


[2.777652508467225, 0.31347241867043846]

In [429]:
model = keras.Sequential()
# model.add(Embedding(vocab_size, 300,input_length=15))
model.add(Dense(300, activation='sigmoid', input_dim=15))
model.add(Dense(300, activation='sigmoid'))
model.add(Dense(14, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

In [430]:
history = model.fit(cc_input_data,cc_input_data_out ,validation_data=(cc_val_data,cc_val_data_out), batch_size=128, epochs=4)

Train on 480706 samples, validate on 16926 samples
Epoch 1/4
480706/480706 [==============================] - 12s 26us/step - loss: 0.3623 - categorical_accuracy: 0.2733 - val_loss: 0.3074 - val_categorical_accuracy: 0.2127
Epoch 2/4
480706/480706 [==============================] - 12s 25us/step - loss: 0.3528 - categorical_accuracy: 0.2822 - val_loss: 0.3024 - val_categorical_accuracy: 0.3311
Epoch 3/4
480706/480706 [==============================] - 12s 25us/step - loss: 0.3464 - categorical_accuracy: 0.2847 - val_loss: 0.3041 - val_categorical_accuracy: 0.3002
Epoch 4/4
480706/480706 [==============================] - 12s 25us/step - loss: 0.3408 - categorical_accuracy: 0.2881 - val_loss: 0.3029 - val_categorical_accuracy: 0.2917


In [431]:
model = keras.Sequential()
# model.add(Embedding(vocab_size, 300,input_length=15))
model.add(Dense(300, activation='sigmoid', input_dim=15))
model.add(Dense(300, activation='sigmoid'))
model.add(Dense(300, activation='sigmoid'))
model.add(Dense(14, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

In [432]:
history = model.fit(cc_input_data,cc_input_data_out ,validation_data=(cc_val_data,cc_val_data_out), batch_size=128, epochs=4)

Train on 480706 samples, validate on 16926 samples
Epoch 1/4
480706/480706 [==============================] - 17s 35us/step - loss: 0.3612 - categorical_accuracy: 0.2776 - val_loss: 0.3047 - val_categorical_accuracy: 0.3257
Epoch 2/4
480706/480706 [==============================] - 16s 33us/step - loss: 0.3472 - categorical_accuracy: 0.2888 - val_loss: 0.3023 - val_categorical_accuracy: 0.2896
Epoch 3/4
480706/480706 [==============================] - 16s 33us/step - loss: 0.3392 - categorical_accuracy: 0.2916 - val_loss: 0.3045 - val_categorical_accuracy: 0.3026
Epoch 4/4
480706/480706 [==============================] - 16s 33us/step - loss: 0.3315 - categorical_accuracy: 0.2933 - val_loss: 0.3047 - val_categorical_accuracy: 0.3212


In [446]:
model = keras.Sequential()
model.add(Embedding(vocab_size, 10,input_length=15))
model.add(Flatten())
model.add(Dense(30, activation='sigmoid'))
model.add(Dense(14, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

In [447]:
history = model.fit(cc_input_data,cc_input_data_out ,validation_data=(cc_val_data,cc_val_data_out), batch_size=256, epochs=4)

Train on 480706 samples, validate on 16926 samples
Epoch 1/4
480706/480706 [==============================] - 9s 18us/step - loss: 0.3121 - categorical_accuracy: 0.3606 - val_loss: 0.2770 - val_categorical_accuracy: 0.3414
Epoch 2/4
480706/480706 [==============================] - 8s 16us/step - loss: 0.1623 - categorical_accuracy: 0.5833 - val_loss: 0.3133 - val_categorical_accuracy: 0.3344
Epoch 3/4
480706/480706 [==============================] - 8s 16us/step - loss: 0.1039 - categorical_accuracy: 0.6364 - val_loss: 0.3685 - val_categorical_accuracy: 0.3245
Epoch 4/4
480706/480706 [==============================] - 8s 16us/step - loss: 0.0760 - categorical_accuracy: 0.6496 - val_loss: 0.4256 - val_categorical_accuracy: 0.3246


In [471]:
model = keras.Sequential()
model.add(Embedding(vocab_size, 10,input_length=15))
model.add(Conv1D(filters=15,kernel_size=2))
model.add(GlobalMaxPooling1D())
model.add(Dense(30, activation='sigmoid'))
model.add(Dense(14, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

In [472]:
history = model.fit(cc_input_data,cc_input_data_out ,validation_data=(cc_val_data,cc_val_data_out), batch_size=256, epochs=4)

Train on 480706 samples, validate on 16926 samples
Epoch 1/4
480706/480706 [==============================] - 13s 28us/step - loss: 0.3132 - categorical_accuracy: 0.3622 - val_loss: 0.2806 - val_categorical_accuracy: 0.3209
Epoch 2/4
480706/480706 [==============================] - 12s 25us/step - loss: 0.1933 - categorical_accuracy: 0.4961 - val_loss: 0.3093 - val_categorical_accuracy: 0.3052
Epoch 3/4
480706/480706 [==============================] - 12s 24us/step - loss: 0.1522 - categorical_accuracy: 0.5294 - val_loss: 0.3411 - val_categorical_accuracy: 0.3040
Epoch 4/4
480706/480706 [==============================] - 12s 24us/step - loss: 0.1292 - categorical_accuracy: 0.5360 - val_loss: 0.3725 - val_categorical_accuracy: 0.2919


In [478]:
model = keras.Sequential()
model.add(Embedding(vocab_size, 10,input_length=15))
model.add(LSTM(units=15))
model.add(Dense(30, activation='sigmoid'))
model.add(Dense(14, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

In [479]:
history = model.fit(cc_input_data,cc_input_data_out ,validation_data=(cc_val_data,cc_val_data_out), batch_size=512, epochs=10)

Train on 480706 samples, validate on 16926 samples
Epoch 1/10
480706/480706 [==============================] - 22s 45us/step - loss: 0.3965 - categorical_accuracy: 0.2432 - val_loss: 0.3013 - val_categorical_accuracy: 0.3406
Epoch 2/10
480706/480706 [==============================] - 20s 41us/step - loss: 0.3287 - categorical_accuracy: 0.3195 - val_loss: 0.2875 - val_categorical_accuracy: 0.3196
Epoch 3/10
480706/480706 [==============================] - 20s 42us/step - loss: 0.2611 - categorical_accuracy: 0.4215 - val_loss: 0.2971 - val_categorical_accuracy: 0.3073
Epoch 4/10
480706/480706 [==============================] - 20s 42us/step - loss: 0.2048 - categorical_accuracy: 0.5103 - val_loss: 0.3144 - val_categorical_accuracy: 0.3185
Epoch 5/10
480706/480706 [==============================] - 20s 41us/step - loss: 0.1678 - categorical_accuracy: 0.5882 - val_loss: 0.3387 - val_categorical_accuracy: 0.3311
Epoch 6/10
480706/480706 [==============================] - 20s 42us/step - los

In [481]:
model = keras.Sequential()
model.add(Embedding(vocab_size, 10,input_length=15))
model.add(LSTM(units=15,dropout=0.5))
model.add(Dense(30, activation='sigmoid'))
model.add(Dense(14, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

In [482]:
history = model.fit(cc_input_data,cc_input_data_out ,validation_data=(cc_val_data,cc_val_data_out), batch_size=512, epochs=10)

Train on 480706 samples, validate on 16926 samples
Epoch 1/10
480706/480706 [==============================] - 24s 51us/step - loss: 0.3896 - categorical_accuracy: 0.2492 - val_loss: 0.2951 - val_categorical_accuracy: 0.2055
Epoch 2/10
480706/480706 [==============================] - 22s 46us/step - loss: 0.3106 - categorical_accuracy: 0.2419 - val_loss: 0.2923 - val_categorical_accuracy: 0.2200
Epoch 3/10
480706/480706 [==============================] - 22s 46us/step - loss: 0.2638 - categorical_accuracy: 0.3352 - val_loss: 0.2947 - val_categorical_accuracy: 0.2900
Epoch 4/10
480706/480706 [==============================] - 22s 45us/step - loss: 0.2280 - categorical_accuracy: 0.4346 - val_loss: 0.2994 - val_categorical_accuracy: 0.2927
Epoch 5/10
480706/480706 [==============================] - 22s 46us/step - loss: 0.1998 - categorical_accuracy: 0.4691 - val_loss: 0.3095 - val_categorical_accuracy: 0.2929
Epoch 6/10
480706/480706 [==============================] - 22s 46us/step - los

In [483]:
model = keras.Sequential()
model.add(Embedding(vocab_size, 10,input_length=15))
model.add(LSTM(units=15,dropout=0.5))
model.add(Dense(14, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=['categorical_accuracy'])

In [484]:
history = model.fit(cc_input_data,cc_input_data_out ,validation_data=(cc_val_data,cc_val_data_out), batch_size=512, epochs=10)

Train on 480706 samples, validate on 16926 samples
Epoch 1/10
480706/480706 [==============================] - 24s 50us/step - loss: 0.3729 - categorical_accuracy: 0.2750 - val_loss: 0.2868 - val_categorical_accuracy: 0.3534
Epoch 2/10
480706/480706 [==============================] - 22s 46us/step - loss: 0.2979 - categorical_accuracy: 0.3678 - val_loss: 0.2798 - val_categorical_accuracy: 0.3122
Epoch 3/10
480706/480706 [==============================] - 22s 45us/step - loss: 0.2621 - categorical_accuracy: 0.3925 - val_loss: 0.2765 - val_categorical_accuracy: 0.2912
Epoch 4/10
480706/480706 [==============================] - 22s 46us/step - loss: 0.2378 - categorical_accuracy: 0.3910 - val_loss: 0.2806 - val_categorical_accuracy: 0.2862
Epoch 5/10
480706/480706 [==============================] - 22s 46us/step - loss: 0.2195 - categorical_accuracy: 0.3993 - val_loss: 0.2868 - val_categorical_accuracy: 0.2821
Epoch 6/10
480706/480706 [==============================] - 21s 45us/step - los

In [485]:
model = keras.Sequential()
model.add(Embedding(vocab_size, 10,input_length=15))
model.add(LSTM(units=100,dropout=0.5))
model.add(Dense(14, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=['categorical_accuracy'])

In [486]:
history = model.fit(cc_input_data,cc_input_data_out ,validation_data=(cc_val_data,cc_val_data_out), batch_size=512, epochs=4)

Train on 480706 samples, validate on 16926 samples
Epoch 1/4
480706/480706 [==============================] - 63s 130us/step - loss: 0.3368 - categorical_accuracy: 0.3093 - val_loss: 0.2784 - val_categorical_accuracy: 0.3377
Epoch 2/4
480706/480706 [==============================] - 60s 125us/step - loss: 0.2544 - categorical_accuracy: 0.3886 - val_loss: 0.2754 - val_categorical_accuracy: 0.3392
Epoch 3/4
480706/480706 [==============================] - 59s 124us/step - loss: 0.2061 - categorical_accuracy: 0.4286 - val_loss: 0.2780 - val_categorical_accuracy: 0.3276
Epoch 4/4
480706/480706 [==============================] - 61s 127us/step - loss: 0.1781 - categorical_accuracy: 0.4674 - val_loss: 0.2845 - val_categorical_accuracy: 0.3838


In [487]:
model = keras.Sequential()
model.add(Embedding(vocab_size, 100,input_length=15))
model.add(LSTM(units=100,dropout=0.5))
model.add(Dense(14, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=['categorical_accuracy'])

In [488]:
history = model.fit(cc_input_data,cc_input_data_out ,validation_data=(cc_val_data,cc_val_data_out), batch_size=512, epochs=4)

Train on 480706 samples, validate on 16926 samples
Epoch 1/4
480706/480706 [==============================] - 131s 273us/step - loss: 0.2407 - categorical_accuracy: 0.4084 - val_loss: 0.2657 - val_categorical_accuracy: 0.3660
Epoch 2/4
480706/480706 [==============================] - 134s 278us/step - loss: 0.1200 - categorical_accuracy: 0.5198 - val_loss: 0.3120 - val_categorical_accuracy: 0.3779
Epoch 3/4
480706/480706 [==============================] - 129s 269us/step - loss: 0.0849 - categorical_accuracy: 0.5433 - val_loss: 0.3533 - val_categorical_accuracy: 0.3933
Epoch 4/4
480706/480706 [==============================] - 123s 256us/step - loss: 0.0670 - categorical_accuracy: 0.5531 - val_loss: 0.3648 - val_categorical_accuracy: 0.3614


In [493]:
len(model.get_weights()[1])

100

In [494]:
model = keras.Sequential()
model.add(Embedding(vocab_size, 100,input_length=15,mask_zero=True))
model.add(LSTM(units=100,dropout=0.5))
model.add(Dense(14, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=['categorical_accuracy'])

In [495]:
history = model.fit(cc_input_data,cc_input_data_out ,validation_data=(cc_val_data,cc_val_data_out), batch_size=512, epochs=4)

Train on 480706 samples, validate on 16926 samples
Epoch 1/4
480706/480706 [==============================] - 144s 299us/step - loss: 0.2603 - categorical_accuracy: 0.3927 - val_loss: 0.2619 - val_categorical_accuracy: 0.3735
Epoch 2/4
480706/480706 [==============================] - 144s 300us/step - loss: 0.1281 - categorical_accuracy: 0.5149 - val_loss: 0.2991 - val_categorical_accuracy: 0.3854
Epoch 3/4
480706/480706 [==============================] - 141s 293us/step - loss: 0.0891 - categorical_accuracy: 0.5421 - val_loss: 0.3305 - val_categorical_accuracy: 0.3654
Epoch 4/4
480706/480706 [==============================] - 142s 294us/step - loss: 0.0699 - categorical_accuracy: 0.5525 - val_loss: 0.3636 - val_categorical_accuracy: 0.3623


In [512]:
def top_3_accuracy(y_true, y_pred):
    return keras.metrics.top_k_categorical_accuracy(y_true, y_pred, k=3)

model = keras.Sequential()
model.add(Embedding(vocab_size, 100,input_length=15,mask_zero=True))
model.add(LSTM(units=100,dropout=0.5))
model.add(Dense(14, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[top_3_accuracy])

In [513]:
history = model.fit(cc_input_data,cc_input_data_out ,validation_data=(cc_val_data,cc_val_data_out), batch_size=512, epochs=4)

Train on 480706 samples, validate on 16926 samples
Epoch 1/4
480706/480706 [==============================] - 155s 323us/step - loss: 0.2645 - top_3_accuracy: 0.6709 - val_loss: 0.2829 - val_top_3_accuracy: 0.6989
Epoch 2/4
480706/480706 [==============================] - 151s 315us/step - loss: 0.0995 - top_3_accuracy: 0.9275 - val_loss: 0.3524 - val_top_3_accuracy: 0.7083
Epoch 3/4
480706/480706 [==============================] - 156s 324us/step - loss: 0.0604 - top_3_accuracy: 0.9550 - val_loss: 0.4098 - val_top_3_accuracy: 0.6998
Epoch 4/4
480706/480706 [==============================] - 151s 314us/step - loss: 0.0439 - top_3_accuracy: 0.9633 - val_loss: 0.4590 - val_top_3_accuracy: 0.7104


In [514]:
def top_3_accuracy(y_true, y_pred):
    return keras.metrics.top_k_categorical_accuracy(y_true, y_pred, k=3)

model = keras.Sequential()
model.add(Embedding(vocab_size, 100,input_length=15,mask_zero=True))
model.add(LSTM(units=100,dropout=0.5))
model.add(Dense(14, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='RMSProp', metrics=[top_3_accuracy])

In [515]:
history = model.fit(cc_input_data,cc_input_data_out ,validation_data=(cc_val_data,cc_val_data_out), batch_size=512, epochs=4)

Train on 480706 samples, validate on 16926 samples
Epoch 1/4
480706/480706 [==============================] - 144s 300us/step - loss: 0.2591 - top_3_accuracy: 0.6832 - val_loss: 0.2614 - val_top_3_accuracy: 0.7131
Epoch 2/4
480706/480706 [==============================] - 140s 292us/step - loss: 0.1269 - top_3_accuracy: 0.8907 - val_loss: 0.3014 - val_top_3_accuracy: 0.7368
Epoch 3/4
480706/480706 [==============================] - 139s 290us/step - loss: 0.0876 - top_3_accuracy: 0.9269 - val_loss: 0.3333 - val_top_3_accuracy: 0.7359
Epoch 4/4
480706/480706 [==============================] - 140s 292us/step - loss: 0.0685 - top_3_accuracy: 0.9408 - val_loss: 0.3611 - val_top_3_accuracy: 0.7363


In [517]:
model.evaluate(cc_test_data,cc_test_data_out)

8475/8475 [==============================] - 2s 203us/step


[0.3741736157664859, 0.7189380530903121]

In [519]:
def top_3_accuracy(y_true, y_pred):
    return keras.metrics.top_k_categorical_accuracy(y_true, y_pred, k=3)

model = keras.Sequential()
model.add(Embedding(vocab_size, 100,input_length=15,mask_zero=True))
model.add(LSTM(units=100,dropout=0.5))
model.add(Dense(14, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='RMSProp', metrics=[top_3_accuracy,'accuracy','categorical_accuracy'])

In [520]:
history = model.fit(cc_input_data,cc_input_data_out ,validation_data=(cc_val_data,cc_val_data_out), batch_size=512, epochs=4)

Train on 480706 samples, validate on 16926 samples
Epoch 1/4
480706/480706 [==============================] - 135s 280us/step - loss: 0.2571 - top_3_accuracy: 0.6866 - acc: 0.8944 - categorical_accuracy: 0.3799 - val_loss: 0.2664 - val_top_3_accuracy: 0.7184 - val_acc: 0.8875 - val_categorical_accuracy: 0.3795
Epoch 2/4
480706/480706 [==============================] - 135s 280us/step - loss: 0.1281 - top_3_accuracy: 0.8894 - acc: 0.9481 - categorical_accuracy: 0.5082 - val_loss: 0.2937 - val_top_3_accuracy: 0.7343 - val_acc: 0.8843 - val_categorical_accuracy: 0.3934
Epoch 3/4
480706/480706 [==============================] - 131s 272us/step - loss: 0.0888 - top_3_accuracy: 0.9259 - acc: 0.9658 - categorical_accuracy: 0.5418 - val_loss: 0.3326 - val_top_3_accuracy: 0.7342 - val_acc: 0.8792 - val_categorical_accuracy: 0.3857
Epoch 4/4
480706/480706 [==============================] - 130s 271us/step - loss: 0.0696 - top_3_accuracy: 0.9397 - acc: 0.9741 - categorical_accuracy: 0.5522 - val_

In [22]:
def top_3_accuracy(y_true, y_pred):
    return keras.metrics.top_k_categorical_accuracy(y_true, y_pred, k=3)

model = keras.Sequential()
model.add(Embedding(vocab_size, 100,input_length=15,mask_zero=True))
model.add(Bidirectional(LSTM(units=100,dropout=0.5,return_sequences=True)))
model.add(Bidirectional(LSTM(units=100,dropout=0.5)))
model.add(Dense(14, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='RMSProp', metrics=[top_3_accuracy,'accuracy','categorical_accuracy'])

In [26]:
filepath="2xBilstm-{epoch:02d}-{val_loss:.2f}-{val_categorical_accuracy:.2f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
callbacks_list = [checkpoint]

In [27]:
history = model.fit(cc_input_data,cc_input_data_out ,validation_data=(cc_val_data,cc_val_data_out), batch_size=1024, epochs=4,callbacks=callbacks_list)

Train on 358484 samples, validate on 49290 samples
Epoch 1/4
358484/358484 [==============================] - 96s 268us/step - loss: 0.1395 - top_3_accuracy: 0.8842 - acc: 0.9412 - categorical_accuracy: 0.5579 - val_loss: 0.1762 - val_top_3_accuracy: 0.8603 - val_acc: 0.9229 - val_categorical_accuracy: 0.5487

Epoch 00001: saving model to 2xBilstm-01-0.18-0.55.hdf5
Epoch 2/4
358484/358484 [==============================] - 96s 268us/step - loss: 0.1228 - top_3_accuracy: 0.9015 - acc: 0.9493 - categorical_accuracy: 0.5747 - val_loss: 0.1603 - val_top_3_accuracy: 0.8911 - val_acc: 0.9318 - val_categorical_accuracy: 0.5681

Epoch 00002: saving model to 2xBilstm-02-0.16-0.57.hdf5
Epoch 3/4
358484/358484 [==============================] - 96s 268us/step - loss: 0.1092 - top_3_accuracy: 0.9138 - acc: 0.9556 - categorical_accuracy: 0.5832 - val_loss: 0.1500 - val_top_3_accuracy: 0.8924 - val_acc: 0.9373 - val_categorical_accuracy: 0.5851

Epoch 00003: saving model to 2xBilstm-03-0.15-0.59.hdf

In [24]:
model.evaluate(x=cc_test_data, y=cc_test_data_out, batch_size=None, verbose=1, sample_weight=None, steps=None)

30237/30237 [==============================] - 35s 1ms/step


[0.18163946719529575,
 0.8345404636703376,
 0.9197624472866543,
 0.5073585342579497]

In [25]:
model.save('50%_categorical.csv')

In [28]:
model.load_weights("./2xBilstm-03-0.15-0.59.hdf5")

In [60]:
model.evaluate(x=cc_test_data, y=cc_test_data_out, batch_size=None, verbose=1, sample_weight=None, steps=None)

27368/27368 [==============================] - 31s 1ms/step


[0.14074560044130624,
 0.8990061385559778,
 0.9414096788787173,
 0.6011400175387314]

In [61]:
pred = model.predict(cc_val_data)

In [62]:
error = cc_val_data_out - pred

In [64]:
import numpy as np

In [65]:
error = np.absolute(error)

In [66]:
error.to_csv('bilstm_error.csv')

In [68]:
!explorer .

In [72]:
len(cc_val_df)/14

2969.5

In [69]:
[error[cat].sum() for cat in cc_types]

[5089.301023368861,
 762.4743669615234,
 2454.0194316451343,
 1656.4140921296591,
 10343.44581935413,
 2204.0897995197242,
 499.975531778603,
 4319.007700273058,
 9885.208288726579,
 11999.036386445297,
 406.4953425794674,
 1292.0463068031086,
 15.940705289792575,
 26.371238619690075]

In [85]:
error_col_sums = [error[cat].sum() for cat in cc_types]
length_per_cat = [len(cc_val_df[cc_val_df[cat]==True]) for cat in cc_types]
for cat,err,leng in zip(cc_types,error_col_sums,length_per_cat):
    print(cat,err/leng)

is_anagram 0.5518053803934577
is_homophone 0.30609167682116556
is_double 0.6011806544941535
is_cryptic 0.6089757691653158
is_contain 0.5840784809618912
is_reverse 0.3757398226252513
is_alternate 0.6820948591795403
is_init 0.7701511591071786
is_delete 0.8669714338472706
is_charade 0.8708205520317365
is_&lit 0.47654788110136853
is_hidden 0.4530316643769665
is_spoonerism 0.09839941536908997
is_palindrome 0.3995642215104557


In [83]:
[len(cc_val_df[cc_val_df[cat]==True]) for cat in cc_types]

[9223,
 2491,
 4082,
 2720,
 17709,
 5866,
 733,
 5608,
 11402,
 13779,
 853,
 2852,
 162,
 66]

In [77]:
error.columns

Index(['is_anagram', 'is_homophone', 'is_double', 'is_cryptic', 'is_contain',
       'is_reverse', 'is_alternate', 'is_init', 'is_delete', 'is_charade',
       'is_&lit', 'is_hidden', 'is_spoonerism', 'is_palindrome'],
      dtype='object')

In [78]:
cc_types

['is_anagram',
 'is_homophone',
 'is_double',
 'is_cryptic',
 'is_contain',
 'is_reverse',
 'is_alternate',
 'is_init',
 'is_delete',
 'is_charade',
 'is_&lit',
 'is_hidden',
 'is_spoonerism',
 'is_palindrome']

In [122]:
query = 'Proceeding smoothly and evenly, judge back observing cases'

In [123]:
inpt = pad_sequences(tokenizer.texts_to_sequences([query]),maxlen=15)

In [124]:
preds = model.predict(inpt)

In [125]:
cc_types

['is_anagram',
 'is_homophone',
 'is_double',
 'is_cryptic',
 'is_contain',
 'is_reverse',
 'is_alternate',
 'is_init',
 'is_delete',
 'is_charade',
 'is_&lit',
 'is_hidden',
 'is_spoonerism',
 'is_palindrome']

In [126]:
for a,b in zip(cc_types,preds[0]):
    print(a,b)

is_anagram 0.00080093107
is_homophone 0.001205507
is_double 0.00011718823
is_cryptic 3.4680426e-05
is_contain 0.9938917
is_reverse 0.0020614178
is_alternate 0.9986004
is_init 2.895232e-06
is_delete 0.01853707
is_charade 0.0053280066
is_&lit 3.5364006e-05
is_hidden 0.00079043105
is_spoonerism 1.4816669e-06
is_palindrome 1.3082043e-05


In [ ]:
def top_3_accuracy(y_true, y_pred):
    return keras.metrics.top_k_categorical_accuracy(y_true, y_pred, k=3)

model = keras.Sequential()
model.add(Embedding(vocab_size, 100,input_length=15,mask_zero=True))
model.add(Bidirectional(LSTM(units=100,dropout=0.5,return_sequences=True)))
model.add(Bidirectional(LSTM(units=100,dropout=0.5)))
model.add(Dense(14, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='RMSProp', metrics=[top_3_accuracy,'accuracy','categorical_accuracy'])